In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
import json
import numpy as np
import pandas as pd

import datetime
import os
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


DATASET_PATH = "resources/games/all_battles.json"
MODEL_EXPORT_PATH = "resources/models/"
SELECTED_PLAYERS = []
DISCOUNT_FACTOR = 0.98
EPOCHS = 20
BATCH_SIZE = 512
N_CORES = 8
LEARNING_RATE = 0.001

os.environ["OMP_NUM_THREADS"] = str(N_CORES)
os.environ["TF_NUM_INTRAOP_THREADS"] = str(N_CORES)
os.environ["TF_NUM_INTEROP_THREADS"] = str(N_CORES)

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.20.0


In [28]:
#cuda fouten verbergen
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"



In [29]:
def build_dataset(path):
    """
    CRITICAL FIX: Generate training data from BOTH players' perspectives.

    The key insight: In a two-player zero-sum game, we need to train the model
    to evaluate positions from a consistent perspective. We do this by:
    1. For each board position, create TWO training examples
    2. One from Light's perspective (original board, positive if Light wins)
    3. One from Dark's perspective (inverted board, positive if Dark wins)
    """
    with open(path) as f:
        data = json.load(f)

    print(f"Processing {len(data)} games")

    boards = []
    scores = []

    for game_idx, game in enumerate(data):
        # Skip games that don't have both players in the selected players list
        if SELECTED_PLAYERS and (
                game["lightPlayer"] not in SELECTED_PLAYERS or
                game["darkPlayer"] not in SELECTED_PLAYERS
        ):
            continue

        winner = game["winner"]  # 1 for Light, -1 for Dark
        n_moves = len(game["boardHistory"])
        reserve_size = game['reserveSize']

        for i, board_as_long in enumerate(game["boardHistory"]):
            # Convert board to array (0 = Light, 1 = Dark)
            board_as_array = np.array(
                [(board_as_long >> j) & 1 for j in range(59, -1, -1)],
                dtype=np.float32
            )

            # Calculate discount based on proximity to end of game
            # Positions closer to the end are more certain
            discount = DISCOUNT_FACTOR ** (n_moves - i - 1)

            # Light's perspective: positive if Light wins
            light_score = winner * discount * reserve_size
            boards.append(board_as_array)
            scores.append(light_score)

            # Dark's perspective: flip the board (0->1, 1->0) and score
            # This teaches the model to evaluate from the active player's view
            dark_board = 1.0 - board_as_array
            dark_score = -winner * discount * reserve_size  # Flip the score
            boards.append(dark_board)
            scores.append(dark_score)

        if game_idx % 1000 == 0:
            print(f"Processed game {game_idx}/{len(data)}")

    boards_array = np.array(boards, dtype=np.float32)
    scores_array = np.array(scores, dtype=np.float32)

    # Shuffle the dataset to mix Light and Dark perspectives
    indices = np.random.permutation(len(boards_array))

    return boards_array[indices], scores_array[indices]



In [30]:
def build_conv1d_model(
        filters1=32,
        filters2=64,
        kernel_size=3,
        dense_units1=64,
        dense_units2=32,
        dropout=0.3,
):
    """
    Conv1D-model dat de 60 velden als sequentie ziet: (60, 1)
    """
    inputs = layers.Input(shape=(60,), dtype=tf.float32)

    x = layers.Reshape((60, 1))(inputs)  # (batch, 60, 1)

    x = layers.Conv1D(filters1, kernel_size=kernel_size,
                      activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv1D(filters2, kernel_size=kernel_size,
                      activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)

    x = layers.Dense(dense_units1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(dense_units2, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model



def build_conv2d_model(
        filters1,
        filters2,
        kernel_size,
        dense_units1,
        dense_units2,
        dropout,
):
    """
    Conv2D-model: we reshapen 60 -> (6, 10, 1).
    Dit is een beetje artificieel, maar laat je 2D-convs testen.
    """
    inputs = layers.Input(shape=(60,), dtype=tf.float32)

    x = layers.Reshape((6, 10, 1))(inputs)  # (batch, 6, 10, 1)

    x = layers.Conv2D(filters1, kernel_size=kernel_size,
                      activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(filters2, kernel_size=kernel_size,
                      activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Flatten()(x)

    x = layers.Dense(dense_units1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(dense_units2, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model



Dense modellen

In [31]:
def build_dense4_model(
        units1=256,
        units2=128,
        units3=64,
        units4=32,
        dropout1=0.3,
        dropout2=0.3,
        dropout3=0.2,
):

    inputs = layers.Input(shape=(60,), dtype=tf.float32)

    x = layers.Dense(units1, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout1)(x)

    x = layers.Dense(units2, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout2)(x)

    x = layers.Dense(units3, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout3)(x)

    x = layers.Dense(units4, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model


def build_dense5_model(
        units1=256,
        units2=192,
        units3=128,
        units4=64,
        units5=32,
        dropout1=0.3,
        dropout2=0.3,
        dropout3=0.2,
        dropout4=0.2,
):
    """
    5-laags MLP voor als je echt diep wilt gaan.
    """
    inputs = layers.Input(shape=(60,), dtype=tf.float32)

    x = layers.Dense(units1, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout1)(x)

    x = layers.Dense(units2, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout2)(x)

    x = layers.Dense(units3, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout3)(x)

    x = layers.Dense(units4, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout4)(x)

    x = layers.Dense(units5, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model
def build_dense6_model(
        units1=384, units2=256, units3=192, units4=128, units5=64, units6=32,
        dropout1=0.3, dropout2=0.3, dropout3=0.3, dropout4=0.2, dropout5=0.2,
):
    inputs = layers.Input(shape=(60,), dtype=tf.float32)

    x = layers.Dense(units1, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout1)(x)

    x = layers.Dense(units2, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout2)(x)

    x = layers.Dense(units3, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout3)(x)

    x = layers.Dense(units4, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout4)(x)

    x = layers.Dense(units5, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout5)(x)

    x = layers.Dense(units6, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [32]:
def create_optimizer(name, learning_rate):
    name = name.lower()
    if name == "adam":
        return tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif name == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    elif name == "rmsprop":
        return tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError(f"Unknown optimizer name: {name}")


In [33]:
def train_single_experiment(
        model_builder,
        model_kwargs,
        optimizer_name,
        learning_rate,
        batch_size,
        X_train, y_train,
        X_val, y_val,
        X_test, y_test,
        label,
        family,
):
    print(f"\n==== Running experiment: {label} ({family}) ====")

    model = model_builder(**model_kwargs)
    opt = create_optimizer(optimizer_name, learning_rate)

    model.compile(
        optimizer=opt,
        loss="mse",
        metrics=["mae"]
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=4,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=2,
            min_lr=1e-6
        )
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=0,
    )

    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

    best_val_loss = float(min(history.history["val_loss"]))
    best_val_mae = float(min(history.history["val_mae"]))

    print(f"Best val_loss: {best_val_loss:.4f}, best val_mae: {best_val_mae:.4f}")
    print(f"Test loss:     {test_loss:.4f}, test MAE:     {test_mae:.4f}")

    result = {
        "label": label,
        "family": family,
        "optimizer": optimizer_name,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "best_val_loss": best_val_loss,
        "best_val_mae": best_val_mae,
        "test_loss": test_loss,
        "test_mae": test_mae,
        "history": history.history,
    }
    return result


In [34]:
def generate_experiments():
    experiments = []

    conv1d_archs = [
        {
            "name": "conv1d_small",
            "family": "conv1d",
            "builder": build_conv1d_model,
            "kwargs": dict(
                filters1=32, filters2=64,
                kernel_size=3,
                dense_units1=64, dense_units2=32,
                dropout=0.3,
            ),
        },
        {
            "name": "conv1d_medium",
            "family": "conv1d",
            "builder": build_conv1d_model,
            "kwargs": dict(
                filters1=64, filters2=128,
                kernel_size=5,
                dense_units1=128, dense_units2=64,
                dropout=0.4,
            ),
        },
        {
            "name": "conv1d_large",
            "family": "conv1d",
            "builder": build_conv1d_model,
            "kwargs": dict(
                filters1=96, filters2=192,
                kernel_size=5,
                dense_units1=128, dense_units2=64,
                dropout=0.4,
            ),
        },
    ]

    conv_optimizers = ["adam", "rmsprop", "sgd"]
    conv_lrs = [0.0005, 0.001]
    conv_batch_sizes = [256, 512]

    for arch in conv1d_archs:
        for opt in conv_optimizers:
            for lr in conv_lrs:
                for bs in conv_batch_sizes:
                    label = f"{arch['name']}_{opt}_lr{lr}_bs{bs}"
                    experiments.append({
                        "label": label,
                        "family": arch["family"],
                        "model_builder": arch["builder"],
                        "model_kwargs": arch["kwargs"],
                        "optimizer": opt,
                        "learning_rate": lr,
                        "batch_size": bs,
                    })

    
    conv2d_archs = [
        {
            "name": "conv2d_small",
            "family": "conv2d",
            "builder": build_conv2d_model,
            "kwargs": dict(
                filters1=32, filters2=64,
                kernel_size=(3, 3),
                dense_units1=64, dense_units2=32,
                dropout=0.3,
            ),
        },
        {
            "name": "conv2d_medium",
            "family": "conv2d",
            "builder": build_conv2d_model,
            "kwargs": dict(
                filters1=64, filters2=128,
                kernel_size=(3, 3),
                dense_units1=128, dense_units2=64,
                dropout=0.3,
            ),
        },
    ]

    conv2_optimizers = ["adam", "rmsprop"]
    conv2_lrs = [0.0005, 0.001]
    conv2_batch_sizes = [256, 512]

    for arch in conv2d_archs:
        for opt in conv2_optimizers:
            for lr in conv2_lrs:
                for bs in conv2_batch_sizes:
                    label = f"{arch['name']}_{opt}_lr{lr}_bs{bs}"
                    experiments.append({
                        "label": label,
                        "family": arch["family"],
                        "model_builder": arch["builder"],
                        "model_kwargs": arch["kwargs"],
                        "optimizer": opt,
                        "learning_rate": lr,
                        "batch_size": bs,
                    })

 
    dense_archs = [
        # DENSE4
        {
            "name": "dense4_small",
            "family": "dense4",
            "builder": build_dense4_model,
            "kwargs": dict(
                units1=128, units2=64, units3=32, units4=16,
                dropout1=0.3, dropout2=0.3, dropout3=0.2,
            ),
        },
        {
            "name": "dense4_big",
            "family": "dense4",
            "builder": build_dense4_model,
            "kwargs": dict(
                units1=256, units2=128, units3=64, units4=32,
                dropout1=0.3, dropout2=0.3, dropout3=0.2,
            ),
        },
        # DENSE5
        {
            "name": "dense5_small",
            "family": "dense5",
            "builder": build_dense5_model,
            "kwargs": dict(
                units1=256, units2=192, units3=128, units4=64, units5=32,
                dropout1=0.3, dropout2=0.3, dropout3=0.2, dropout4=0.2,
            ),
        },
        {
            "name": "dense5_big",
            "family": "dense5",
            "builder": build_dense5_model,
            "kwargs": dict(
                units1=384, units2=256, units3=192, units4=96, units5=48,
                dropout1=0.3, dropout2=0.3, dropout3=0.2, dropout4=0.2,
            ),
        },
        # DENSE6
        {
            "name": "dense6_small",
            "family": "dense6",
            "builder": build_dense6_model,
            "kwargs": dict(
                units1=256, units2=192, units3=128, units4=96, units5=64, units6=32,
                dropout1=0.3, dropout2=0.3, dropout3=0.3, dropout4=0.2, dropout5=0.2,
            ),
        },
        {
            "name": "dense6_big",
            "family": "dense6",
            "builder": build_dense6_model,
            "kwargs": dict(
                units1=384, units2=256, units3=192, units4=128, units5=64, units6=32,
                dropout1=0.3, dropout2=0.3, dropout3=0.3, dropout4=0.2, dropout5=0.2,
            ),
        },
    ]

    dense_optimizers = ["adam", "rmsprop"]
    dense_lrs = [0.001]
    dense_batch_sizes = [256, 512]

    for arch in dense_archs:
        for opt in dense_optimizers:
            for lr in dense_lrs:
                for bs in dense_batch_sizes:
                    label = f"{arch['name']}_{opt}_lr{lr}_bs{bs}"
                    experiments.append({
                        "label": label,
                        "family": arch["family"],
                        "model_builder": arch["builder"],
                        "model_kwargs": arch["kwargs"],
                        "optimizer": opt,
                        "learning_rate": lr,
                        "batch_size": bs,
                    })

    print(f"Generated {len(experiments)} experiments.")
    return experiments


some helpers and plots

In [35]:
def select_top_by_family(results, family, metric="best_val_mae", top_n=3):
    family_results = [r for r in results if r["family"] == family]
    family_results_sorted = sorted(family_results, key=lambda r: r[metric])
    return family_results_sorted[:top_n]


plotting everything per fammily

In [36]:
def plot_family_histories(results, family, metric="mae", top_n=5):
    top_results = select_top_by_family(results, family, metric="best_val_mae", top_n=top_n)

    if not top_results:
        print(f"No results for family={family}")
        return

    plt.figure()
    for r in top_results:
        h = r["history"]
        label = r["label"]
        if metric in h:
            plt.plot(h[metric], linestyle="-", alpha=0.7, label=f"{label} (train)")
        val_key = f"val_{metric}"
        if val_key in h:
            plt.plot(h[val_key], linestyle="--", alpha=0.7, label=f"{label} (val)")

    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.title(f"{family} – top {top_n} by val MAE ({metric} per epoch)")
    plt.legend(fontsize=7)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


best inter familly

In [37]:
def plot_best_overall(results, metric="mae"):
    best_per_family = []
    families = sorted(set(r["family"] for r in results))
    for fam in families:
        top = select_top_by_family(results, fam, metric="best_val_mae", top_n=1)
        if top:
            best_per_family.append(top[0])

    if not best_per_family:
        print("No results to plot in best_overall.")
        return

    plt.figure()
    for r in best_per_family:
        h = r["history"]
        label = f"{r['family']} | {r['label']}"
        val_key = f"val_{metric}"
        if val_key in h:
            plt.plot(h[val_key], linestyle="-", label=label)

    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.title(f"Best model per family – validation {metric}")
    plt.legend(fontsize=7)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [38]:
def run_experiments(X_train, y_train, X_val, y_val, X_test, y_test):
    experiments = generate_experiments()
    print(f"Running {len(experiments)} experiments...\n")

    results = []

    for i, exp in enumerate(experiments):
        print(f"\n=== Experiment {i+1}/{len(experiments)}: {exp['label']} ===")
        res = train_single_experiment(
            model_builder=exp["model_builder"],
            model_kwargs=exp["model_kwargs"],
            optimizer_name=exp["optimizer"],
            learning_rate=exp["learning_rate"],
            batch_size=exp["batch_size"],
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test,
            label=exp["label"],
            family=exp["family"],
        )
        results.append(res)

    # DataFrame overzicht
    df = pd.DataFrame([
        {
            "label": r["label"],
            "family": r["family"],
            "optimizer": r["optimizer"],
            "lr": r["learning_rate"],
            "batch_size": r["batch_size"],
            "best_val_loss": r["best_val_loss"],
            "best_val_mae": r["best_val_mae"],
            "test_loss": r["test_loss"],
            "test_mae": r["test_mae"],
        }
        for r in results
    ])

    print("\n==================== RESULT TABLE (sorted by test MAE) ====================")
    print(df.sort_values("test_mae").head(20))

    # 1) conv1d verschillen
    plot_family_histories(results, family="conv1d", metric="mae", top_n=5)

    # 2) conv2d verschillen
    plot_family_histories(results, family="conv2d", metric="mae", top_n=5)

    # 3) dense4 / dense5 / dense6 verschillen
    plot_family_histories(results, family="dense4", metric="mae", top_n=5)
    plot_family_histories(results, family="dense5", metric="mae", top_n=5)
    plot_family_histories(results, family="dense6", metric="mae", top_n=5)

    # 4) beste per family tegen elkaar
    plot_best_overall(results, metric="mae")

    return results, df


In [ ]:
print("TensorFlow version:", tf.__version__)
print("Loading dataset...")

boards, scores = build_dataset(DATASET_PATH)

# ---- Train / Val / Test split ----
N = len(boards)
test_size = int(N * 0.2)
val_size  = int((N - test_size) * 0.2)

X_test = boards[:test_size]
y_test = scores[:test_size]

X_trainval = boards[test_size:]
y_trainval = scores[test_size:]

X_val = X_trainval[:val_size]
y_val = y_trainval[:val_size]

X_train = X_trainval[val_size:]
y_train = y_trainval[val_size:]

print(f"Train size: {len(X_train)}")
print(f"Val size:   {len(X_val)}")
print(f"Test size:  {len(X_test)}")

# hyperparameter search
results, df = run_experiments(X_train, y_train, X_val, y_val, X_test, y_test)

# eventueel: beste config tonen
best_row = df.sort_values("test_mae").iloc[0]
print("\nBest experiment based on test MAE:")
print(best_row)

print(df)
print (results)


TensorFlow version: 2.20.0
Loading dataset...
Processing 12000 games
Processed game 0/12000
Processed game 1000/12000
Processed game 2000/12000
Processed game 3000/12000
Processed game 4000/12000
Processed game 5000/12000
Processed game 6000/12000
Processed game 7000/12000
Processed game 8000/12000
Processed game 9000/12000
Processed game 10000/12000
Processed game 11000/12000
Train size: 581664
Val size:   145415
Test size:  181769
Generated 76 experiments.
Running 76 experiments...


=== Experiment 1/76: conv1d_small_adam_lr0.0005_bs256 ===

==== Running experiment: conv1d_small_adam_lr0.0005_bs256 (conv1d) ====


Als train <<< val → overfitting
Als train ≈ val → goed model
Als train ≈ hoog en val ook hoog → underfitting